<a href="https://colab.research.google.com/github/peisuke/generative_ai_notebooks/blob/main/03_StripedHyena.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ninja tokenizers==0.14.1 einops transformers==4.34.1 accelerate flash_attn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 63.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 72.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 93.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.7 MB/s eta 0:00:00
  Created wheel for flash_attn: filename=flash_attn-2.3.6-cp310-cp310-linux_x86_64.whl size=56598219 sha256=07a5c68948f0dbdc00e4ca74e7247ceb0c4adcde7f29d3de358c8ac2b8848709
  Stored in directory: /root/.cache/pip/wheels/24/5f/16/5044cdddb6dfb3331dfbffa28ab6096ec2900777af5cb0253a
Successfully built flash_attn
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface

In [ ]:
# リポジトリのクローン
!git clone https://github.com/togethercomputer/stripedhyena
%cd stripedhyena

Cloning into 'stripedhyena'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 60 (delta 23), reused 39 (delta 11), pack-reused 0
Receiving objects: 100% (60/60), 583.14 KiB | 2.18 MiB/s, done.
Resolving deltas: 100% (23/23), done.
/content/stripedhyena


In [ ]:
!pip install git+https://github.com/HazyResearch/flash-attention.git#subdirectory=csrc/layer_norm

  Cloning https://github.com/HazyResearch/flash-attention.git to /tmp/pip-req-build-vt3clufi
  Running command git clone --filter=blob:none --quiet https://github.com/HazyResearch/flash-attention.git /tmp/pip-req-build-vt3clufi
  Resolved https://github.com/HazyResearch/flash-attention.git to commit 9356a1c0389660d7e231ff3163c1ac17d9e3824a
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... done
  Created wheel for dropout-layer-norm: filename=dropout_layer_norm-0.1-cp310-cp310-linux_x86_64.whl size=152882891 sha256=c538691364c15f6e42aecf26dde9fb6ee0df7a0407fb9cd3c3d4fcf25242ac79
  Stored in directory: /tmp/pip-ephem-wheel-cache-bm26wvzk/wheels/31/f8/df/c88eac07f3197121b3ff88c939d9d7fdcf580291846a645a9f
Successfully built dropout-layer-norm


In [ ]:
from huggingface_hub import snapshot_download
snapshot_download(repo_id="togethercomputer/StripedHyena-Nous-7B")

Fetching 21 files:   0%|          | 0/21 [00:00<?, ?it/s]

'/root/.cache/huggingface/hub/models--togethercomputer--StripedHyena-Nous-7B/snapshots/42777970d603597dadb768705896533eb9556a07'

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer, AutoConfig

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    "togethercomputer/StripedHyena-Nous-7B",
    model_max_length=64,
    trust_remote_code=True,
)
tokenizer.pad_token = tokenizer.eos_token

config = AutoConfig.from_pretrained(
    "togethercomputer/StripedHyena-Nous-7B", trust_remote_code=True
)

config.use_cache = True
device = torch.device("cuda")

model = (
    AutoModelForCausalLM.from_pretrained(
        "togethercomputer/StripedHyena-Nous-7B",
        config=config,
        trust_remote_code=True,
    )
    .to(device)
)

A new version of the following files was downloaded from https://huggingface.co/togethercomputer/StripedHyena-Nous-7B:
- engine.py
- cache.py
- layers.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/togethercomputer/StripedHyena-Nous-7B:
- utils.py
- model.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
prompt_text = "機械学習とは？"

In [ ]:
%env LC_ALL "en_US.UTF-8"
%env LD_LIBRARY_PATH "/usr/lib64-nvidia"
%env LIBRARY_PATH "/usr/local/cuda/lib64/stubs"

env: LC_ALL="en_US.UTF-8"
env: LD_LIBRARY_PATH="/usr/lib64-nvidia"
env: LIBRARY_PATH="/usr/local/cuda/lib64/stubs"


In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!ldconfig /usr/lib64-nvidia

/bin/bash: warning: setlocale: LC_ALL: cannot change locale ("en_US.UTF-8")
/sbin/ldconfig.real: /usr/local/lib/libtbbbind.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbb.so.12 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc_proxy.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_5.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_0.so.3 is not a symbolic link



In [ ]:
input_ids = tokenizer(prompt_text, return_tensors="pt").input_ids.to(device)

streamer = TextStreamer(tokenizer)

model.generate(
    input_ids,
    max_new_tokens=128,
    temperature=0.99,
    top_k=50,
    top_p=0.95,
    do_sample=True,
    streamer=streamer,
    eos_token_id=tokenizer.eos_token_id,
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Initializing inference params...


RuntimeError: ignored